In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [4]:
# Pregunta: ¿Qué pacientes tienen más de una admisión?
q1 = """
SELECT p.external_id, p.full_name, COUNT(a.hadm_id) as n_admissions
FROM patients p
JOIN admissions a ON p.subject_id = a.subject_id
GROUP BY p.subject_id, p.external_id, p.full_name
HAVING COUNT(a.hadm_id) > 1;
"""
pd.read_sql(q1, engine)

,external_id,full_name,n_admissions
0,MRN-0006,Luc??a Herrera,2
1,MRN-0001,Ana L??pez,2


In [5]:
# Pregunta: ¿Cuánto duró cada admisión?
q2 = """
SELECT a.hadm_id, p.full_name,
EXTRACT(EPOCH FROM (a.dischtime - a.admittime)) / 86400.0 AS length_of_stay_days
FROM admissions a
JOIN patients p ON a.subject_id = p.subject_id
ORDER BY length_of_stay_days DESC;
"""
pd.read_sql(q2, engine)

,hadm_id,full_name,length_of_stay_days
0,5,Jos?? Mart??nez,8.375000
1,3,Carlos P??rez,7.500000
2,9,Miguel Castillo,6.333333
3,1,Ana L??pez,5.250000
4,2,Ana L??pez,3.958333
5,6,Alex Rivera,3.541667
6,7,Luc??a Herrera,3.020833
7,4,Mar??a G??mez,2.916667
8,8,Luc??a Herrera,2.833333
9,10,Sof??a Morales,2.541667


In [6]:
# Pregunta: Creatinina máxima por admisión
q3 = """
SELECT a.hadm_id, p.full_name, MAX(l.value_num) as max_creatinine
FROM labevents l
JOIN d_labitems d ON l.labitem_id = d.labitem_id
JOIN admissions a ON l.hadm_id = a.hadm_id
JOIN patients p ON a.subject_id = p.subject_id
WHERE d.label = 'Creatinine'
GROUP BY a.hadm_id, p.external_id, p.full_name;
"""
pd.read_sql(q3, engine)

,hadm_id,full_name,max_creatinine
0,1,Ana L??pez,1.6
1,3,Carlos P??rez,3.1
2,7,Luc??a Herrera,0.9
3,8,Luc??a Herrera,1.4
4,9,Miguel Castillo,1.8


In [7]:
# Pregunta: Casos sospechosos de Sepsis (WBC alto o Lactato alto)
q4 = """
SELECT a.hadm_id, p.full_name,
MAX(CASE WHEN d.label = 'White Blood Cells' THEN l.value_num END) as max_wbc,
MAX(CASE WHEN d.label = 'Lactate' THEN l.value_num END) as max_lactate
FROM labevents l
JOIN d_labitems d ON l.labitem_id = d.labitem_id
JOIN admissions a ON l.hadm_id = a.hadm_id
JOIN patients p ON a.subject_id = p.subject_id
WHERE d.label IN ('White Blood Cells', 'Lactate')
GROUP BY a.hadm_id, p.external_id, p.full_name
HAVING MAX(CASE WHEN d.label = 'White Blood Cells' THEN l.value_num END) > 12 
    OR MAX(CASE WHEN d.label = 'Lactate' THEN l.value_num END) > 2;
"""
pd.read_sql(q4, engine)

,hadm_id,full_name,max_wbc,max_lactate
0,3,Carlos P??rez,18.4,3.8
1,5,Jos?? Mart??nez,14.2,2.2
2,6,Alex Rivera,12.5,NaN
3,10,Sof??a Morales,13.0,NaN
